In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#set up executable path for headless browser
executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
#test url with epic fantasy
url = 'https://www.goodreads.com/list/show/50.The_Best_Epic_Fantasy_fiction_'
browser.visit(url)

In [4]:
html = browser.html
blurb_soup = BeautifulSoup(html, 'html.parser')

link_test = browser.find_link_by_partial_href('/book/show/')[0]

In [5]:
link_test.click()


In [6]:
new_url = browser.url
html = browser.html
blurb_soup = BeautifulSoup(html, 'html.parser')

In [7]:
div = blurb_soup.find('div', class_ = 'readable stacked')
div.find_all('span')[1].text

'Here is the first volume in George R. R. Martin’s magnificent cycle of novels that includes A Clash of Kings and A Storm of Swords. As a whole, this series comprises a genuine masterpiece of modern fantasy, bringing together the best the genre has to offer. Magic, mystery, intrigue, romance, and adventure fill these pages and transport us to a world unlike any we have ever experienced. Already hailed as a classic, George R. R. Martin’s stunning series is destined to stand as one of the great achievements of imaginative fiction.A GAME OF THRONESLong ago, in a time forgotten, a preternatural event threw the seasons out of balance. In a land where summers can last decades and winters a lifetime, trouble is brewing. The cold is returning, and in the frozen wastes to the north of Winterfell, sinister and supernatural forces are massing beyond the kingdom’s protective Wall. At the center of the conflict lie the Starks of Winterfell, a family as harsh and unyielding as the land they were bor

NameError: name 'blurb_scrape' is not defined

In [23]:
# set params
def blurb_scrape(url):
    i = 0
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path)
    for i in range(0, 200, 2):
        #open browser and navigate list
        
        try:
            browser.visit(url)
        
            link = browser.find_link_by_partial_href('/book/show/')[i]
            link.click()
            #need this for some reason otherwise it prints
            html = browser.html
            blurb_soup = BeautifulSoup(html, 'html.parser')
            div = blurb_soup.find('div', class_ = 'readable stacked')
            title = blurb_soup.find('h1').text
            try:
                
            
                blurb = div.find_all('span')[1].text
                print(title)
                print(blurb)
                print("----------------------------------------------------------")
            

            except IndexError:
                
                blurb = div.find_all('span')[0].text
                print(title)
                print(blurb)
                print('----------------------------------------------------------')

SyntaxError: unexpected EOF while parsing (<ipython-input-23-b75ce9b1ce26>, line 33)

In [49]:
def blurb_scrape(url):
    i = 0
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path)
    for i in range(0, 200, 2):
        browser.visit(url)
        link = browser.find_link_by_partial_href('/book/show/')[i]
        link.click()
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        div = soup.find('div', class_ = 'readable stacked')
        title = soup.find('h1').text
        author = soup.find('div', class_ = 'authorName__container').text
        blurb = div.find_all('span')
        if len(blurb) == 2:
            print(title)
            print(f"By: {author}")
            print(blurb[1].text)
            print('----------')
            browser.back()
        elif len(blurb) == 1:
            print(title)
            print(f"By: {author}")
            print(blurb[0].text)
            print('----------')
            browser.back()
        else:
            browser.back()
                
        

In [47]:
blurb_scrape('https://www.goodreads.com/list/show/135.Best_Horror_Novels')

KeyboardInterrupt: 

In [ ]:
def blurb_scrape(url):
    i = 0
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path)
    for i in range(0, 200, 2):
        browser.visit(url)
        link = browser.find_link_by_partial_href('/book/show/')[i]
        link.click()
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        div = soup.find('div', class_ = 'readable stacked')
        title = soup.find('h1').text
        author = soup.find('div', class_ = 'authorName__container').text
        
        try:
            blurb = div.find_all('span')[1].text
            print(title)
            print(f"By: {author}")
            print(blurb)
            print('----------')
            
        except IndexError:
            blurb = div.find_all('span')[0].text
            print(title)
            print(f"By: {author}")
            print(blurb)
            print('----------')
        browser.back()

In [40]:
def loop_scrape(url):
    blurb_scrape(url)
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path)
    browser.visit(url)
    browser.is_element_present_by_text('Next', wait_time = 3)
    next_elem = browser.find_link_by_partial_text('Next').first
    next_elem.click()
    new_url = browser.url
    browser.quit()
    count = 0
    while (count < 100):
        try:
            blurb_scrape(new_url)
            executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
            browser = Browser('chrome', **executable_path)
            browser.visit(new_url)
        
            browser.is_element_present_by_text('Next', wait_time = 3)
            next_elem = browser.find_link_by_partial_text('Next').first
            next_elem.click()
            new_url = browser.url
            browser.quit()
            count += 1
        except:
            print("Something Happened!")
  
        

In [45]:
loop_scrape('https://www.goodreads.com/list/show/4893.Best_Science_Fiction_of_the_21st_Century')


      Ready Player One

By: 
Ernest Cline (Goodreads Author)

IN THE YEAR 2044, reality is an ugly place. The only time teenage Wade Watts really feels alive is when he's jacked into the virtual utopia known as the OASIS. Wade's devoted his life to studying the puzzles hidden within this world's digital confines, puzzles that are based on their creator's obsession with the pop culture of decades past and that promise massive power and fortune to whoever can unlock them. But when Wade stumbles upon the first clue, he finds himself beset by players willing to kill to take this ultimate prize. The race is on, and if Wade's going to survive, he'll have to win—and confront the real world he's always been so desperate to escape.
----------

      The Martian

By: 
Andy Weir (Goodreads Author)

Six days ago, astronaut Mark Watney became one of the first people to walk on Mars. Now, he’s sure he’ll be the first person to die there.After a dust storm nearly kills him and forces his crew to eva


      The Road

By: 
Cormac McCarthy

A searing, postapocalyptic novel destined to become Cormac McCarthy’s masterpiece.A father and his son walk alone through burned America. Nothing moves in the ravaged landscape save the ash on the wind. It is cold enough to crack stones, and when the snow falls it is gray. The sky is dark. Their destination is the coast, although they don’t know what, if anything, awaits them there. They have nothing; just a pistol to defend themselves against the lawless bands that stalk the road, the clothes they are wearing, a cart of scavenged food—and each other.The Road is the profoundly moving story of a journey. It boldly imagines a future in which no hope remains, but in which the father and his son, “each the other’s world entire,” are sustained by love. Awesome in the totality of its vision, it is an unflinching meditation on the worst and the best that we are capable of: ultimate destructiveness, desperate tenacity, and the tenderness that keeps two pe


      Revelation Space

By: 
Alastair Reynolds

Nine hundred thousand years ago, something annihilated the Amarantin civilization just as it was on the verge of discovering space flight. Now one scientist, Dan Sylveste, will stop at nothing to solve the Amarantin riddle before ancient history repeats itself. With no other resources at his disposal, Sylveste forges a dangerous alliance with the cyborg crew of the starship Nostalgia for Infinity. But as he closes in on the secret, a killer closes in on him. Because the Amarantin were destroyed for a reason — and if that reason is uncovered, the universe—and reality itself — could be irrecoverably altered….
----------

      Oryx and Crake

By: 
Margaret Atwood (Goodreads Author)

Oryx and Crake is at once an unforgettable love story and a compelling vision of the future. Snowman, known as Jimmy before mankind was overwhelmed by a plague, is struggling to survive in a world where he may be the last human, and mourning the loss of his bes


      Catching Fire

By: 
Suzanne Collins

Sparks are igniting.Flames are spreading.And the Capitol wants revenge.Against all odds, Katniss has won the Hunger Games. She and fellow District 12 tribute Peeta Mellark are miraculously still alive. Katniss should be relieved, happy even. After all, she has returned to her family and her longtime friend, Gale. Yet nothing is the way Katniss wishes it to be. Gale holds her at an icy distance. Peeta has turned his back on her completely. And there are whispers of a rebellion against the Capitol - a rebellion that Katniss and Peeta may have helped create.Much to her shock, Katniss has fueled an unrest she's afraid she cannot stop. And what scares her even more is that she's not entirely convinced she should try. As time draws near for Katniss and Peeta to visit the districts on the Capitol's cruel Victory Tour, the stakes are higher than ever. If they can't prove, without a shadow of a doubt, that they are lost in their love for each other, t


      Seveneves

By: 
Neal Stephenson (Goodreads Author)

What would happen if the world were ending?A catastrophic event renders the earth a ticking time bomb. In a feverish race against the inevitable, nations around the globe band together to devise an ambitious plan to ensure the survival of humanity far beyond our atmosphere, in outer space.But the complexities and unpredictability of human nature coupled with unforeseen challenges and dangers threaten the intrepid pioneers, until only a handful of survivors remain . . .Five thousand years later, their progeny—seven distinct races now three billion strong—embark on yet another audacious journey into the unknown . . . to an alien world utterly transformed by cataclysm and time: Earth.A writer of dazzling genius and imaginative vision, Neal Stephenson combines science, philosophy, technology, psychology, and literature in a magnificent work of speculative fiction that offers a portrait of a future that is both extraordinary and eer

...


      Brass Man

By: 
Neal Asher (Goodreads Author)

Ian Cormac, a legendary Earth Central Security agent, the James Bond of a wealthy future, is hunting an interstellar dragon, little knowing that, far away, his competition has resurrected an horrific killing machine named "Mr. Crane" to assist in a similar hunt, ecompassing whole star systems. Mr. Crane, the insane indestructible artificial man now in a new metal body, seeks to escape a bloody past he can neither forget nor truly remember. And he is on a collision course with Ian Cormac.
----------

      Trading in Danger

By: 
Elizabeth Moon

Kylara Vatta is the only daughter in a family full of sons, and her father’s only child to buck tradition by choosing a military career instead of joining the family business. For Ky, it’s no contest: Even running the prestigious Vatta Transport Ltd. shipping concern can’t hold a candle to shipping out as an officer aboard an interstellar cruiser. It’s adventure, not commerce, that stirs her 


      Seeker

By: 
Jack McDevitt

The Barnes & Noble ReviewAlex Benedict and his executive assistant, Chase Kolpath -- ambitious antiquities dealers from Jack McDevitt's A Talent for War (1989) and, more recently, Polaris -- are back in Seeker, a story in which the two antiquarians search for a legendary lost colony that is both a science fiction thriller and a remarkably complex mystery. More than 9,000 years after an interstellar transport named Seeker left an overcrowded and politically repressive Earth with the dream of founding a new society on an unspecified planet, Benedict and Kolpath stumble across a ceramic cup that was once on the now-legendary lost starship. But tracking down how the ancient artifact got from the ship into the hands of a maltreated woman and her thuggish boyfriend turns out to be more than Benedict and Kolpath bargained for -- as their search leads them across multiple star systems and straight into an anonymous assassin's crosshairs. But as the killer clo


      The Line Of Polity

By: 
Neal Asher (Goodreads Author)

Outlink station Miranda has been destroyed by a nanomycelium and the very nature of this sabotage suggests that the alien bioconstruct Dragon - a creature as untrustworthy as it is gigantic - is somehow involved.
----------

      Darwin's Children

By: 
Greg Bear

Greg Bear’s Nebula Award–winning novel, Darwin’s Radio, painted a chilling portrait of humankind on the threshold of a radical leap in evolution—one that would alter our species forever. Now Bear continues his provocative tale of the human race confronted by an uncertain future, where “survival of the fittest” takes on astonishing and controversial new dimensions.DARWIN’S CHILDRENEleven years have passed since SHEVA, an ancient retrovirus, was discovered in human DNA—a retrovirus that caused mutations in the human genome and heralded the arrival of a new wave of genetically enhanced humans. Now these changed children have reached adolescence . . . and face a worl


      Saturn's Children

By: 
Charles Stross (Goodreads Author)

Sometime in the twenty-third century, humanity went extinct—leaving only androids behind. Freya Nakamichi 47 is a femmebot, one of the last of her kind still functioning. With no humans left to pay for the pleasures she provides, she agrees to transport a mysterious package from Mercury to Mars. Unfortunately for Freya, she has just made herself a moving target for some very powerful, very determined humanoids who will stop at nothing to possess the contents of the package.
----------

      Shadow Puppets

By: 
Orson Scott Card

Bestselling author Orson Scott Card brings to life a new chapter in the saga of Ender's Earth.Earth and its society have been changed irrevocably in the aftermath of Ender Wiggin's victory over the Formics. The unity forced upon the warring nations by an alien enemy has shattered. Nations are rising again, seeking territory and influence, and most of all, seeking to control the skills and loyalt

NameError: name 'ElementDoesNotExistError' is not defined

In [50]:
blurb_scrape('https://www.goodreads.com/list/show/50.The_Best_Epic_Fantasy_fiction_')


      A Game of Thrones

By: 
George R.R. Martin

Here is the first volume in George R. R. Martin’s magnificent cycle of novels that includes A Clash of Kings and A Storm of Swords. As a whole, this series comprises a genuine masterpiece of modern fantasy, bringing together the best the genre has to offer. Magic, mystery, intrigue, romance, and adventure fill these pages and transport us to a world unlike any we have ever experienced. Already hailed as a classic, George R. R. Martin’s stunning series is destined to stand as one of the great achievements of imaginative fiction.A GAME OF THRONESLong ago, in a time forgotten, a preternatural event threw the seasons out of balance. In a land where summers can last decades and winters a lifetime, trouble is brewing. The cold is returning, and in the frozen wastes to the north of Winterfell, sinister and supernatural forces are massing beyond the kingdom’s protective Wall. At the center of the conflict lie the Starks of Winterfell, a family


      Eragon

By: 
Christopher Paolini (Goodreads Author)

One boy...One dragon...A world of adventure. When Eragon finds a polished blue stone in the forest, he thinks it is the lucky discovery of a poor farm boy; perhaps it will buy his family meat for the winter. But when the stone brings a dragon hatchling, Eragon soon realizes he has stumbled upon a legacy nearly as old as the Empire itself. Overnight his simple life is shattered, and he is thrust into a perilous new world of destiny, magic, and power. With only an ancient sword and tge advice of an old storyteller for guidance, Eragon and the fledgling dragon must navigate the dangerous terrain and dark enemies of an Empire ruled by a king whose evil knows no bounds. Can Eragon take up the mantle of the legendary Dragon Riders? The fate of the Empire may rest in his hands.
----------

      Assassin's Apprentice

By: 
Robin Hobb (Goodreads Author)

In a faraway land where members of the royal family are named for the virtues the


      The Belgariad Boxed Set: Pawn of Prophecy / Queen of Sorcery / Magician's Gambit / Castle of Wizardry / Enchanters' End Game

By: 
David Eddings

It all begins with the theft of the Orb that for so long protected the West from an evil god. As long as the Orb was at Riva, the prophecy went, its people would be safe from this corrupting power. Garion, a simple farm boy, is familiar with the legend of the Orb, but skeptical in matters of magic. Until, through a twist of fate, he learns not only that the story of the Orb is true, but that he must set out on a quest of unparalleled magic and danger to help recover it. For Garion is a child of destiny, and fate itself is leading him far from his home, sweeping him irrevocably toward a distant tower-and a cataclysmic confrontation with a master of the darkest magic. The quest may be nearing its end, but the danger continues. After discovering a shocking secret about himself he never could have imagined-all in pursuit of the legendary O


      Graceling

By: 
Kristin Cashore

Katsa has been able to kill a man with her bare hands since she was eight—she’s a Graceling, one of the rare people in her land born with an extreme skill. As niece of the king, she should be able to live a life of privilege, but Graced as she is with killing, she is forced to work as the king’s thug. She never expects to fall in love with beautiful Prince Po. She never expects to learn the truth behind her Grace—or the terrible secret that lies hidden far away . . . a secret that could destroy all seven kingdoms with words alone. With elegant, evocative prose and a cast of unforgettable characters, debut author Kristin Cashore creates a mesmerizing world, a death-defying adventure, and a heart-racing romance that will consume you, hold you captive, and leave you wanting more.
----------

      The Color of Magic

By: 
Terry Pratchett

Terry Pratchett's profoundly irreverent, bestselling novels have garnered him a revered position in the halls of


      The Earthsea Trilogy

By: 
Ursula K. Le Guin

As long ago as forever and as far away as Selidor, there lived the dragonlord and Archmage, Sparrowhawk, the greatest of the great wizards - he who, when still a youth, met with the evil shadow-beast; he who later brought back the Ring of Erreth-Akbe from the Tombs of Atuan; and he who, as an old man, rode the mighty dragon Kalessin back from the land of the dead. And then, the legends say, Sparrowhawk entered his boat, Lookfar, turned his back on land, and without wind or sail or oar moved westward over the sea and out of sight.Wizard of Earthsea, The Tombs of Atuan, The Farthest Shore - Ursula Le Guin's brilliant and magical trilogy.Cover Illustration: Jonathan Field
----------

      Words of Radiance

By: 
Brandon Sanderson (Goodreads Author)

From #1 New York Times bestselling author Brandon Sanderson, Words of Radiance, Book Two of the Stormlight Archive, continues the immersive fantasy epic that The Way of Kings began.Expected

...


      The Magicians' Guild

By: 
Trudi Canavan

"We should expect this young woman to be more powerful than our average novice, possibly even more powerful than the average magician."This year, like every other, the magicians of Imardin gather to purge the city of undesirables. Cloaked in the protection of their sorcery, they move with no fear of the vagrants and miscreants who despise them and their work-—until one enraged girl, barely more than a child, hurls a stone at the hated invaders...and effortlessly penetrates their magical shield.What the Magicians' Guild has long dreaded has finally come to pass. There is someone outside their ranks who possesses a raw power beyond imagining, an untrained mage who must be found and schooled before she destroys herself and her city with a force she cannot yet control.
----------

      The Crystal Cave

By: 
Mary  Stewart

Fifth century Britain is a country of chaos and division after the Roman withdrawal. This is the world of young Merlin,


      The Deed of Paksenarrion

By: 
Elizabeth Moon

The Deed of Paksenarrion revolves around the life of Paksenarrion Dorthansdotter, known as Paks. It takes place in a fictional medieval world comprised of kingdoms of humans, dwarves, and elves. The story begins by introducing Paks as a headstrong girl of 18, who leaves her home (fleeing a marriage arranged by her father) to join a mercenary company. Through her journeys and hardships she comes to realize that she has been gifted as a paladin. The novel was originally published in three volumes in 1988 and 1989 and as a single trade edition of that name in 1992. The three books included are The Sheepfarmer's Daughter, Divided Allegiance and Oath of Gold.From publisher Baen: "Paksenarrion, a simple sheepfarmer's daughter, yearns for a life of adventure and glory, such as was known to heroes in songs and story. At age seventeen she runs away from home to join a mercenary company and begins her epic life . . . Book One: Paks is trained


      Wild Magic

By: 
Tamora Pierce (Goodreads Author)

Young Daine's knack with horses gets her a job helping the royal horsemistress drive a herd of ponies to Tortall. Soon it becomes clear that Daine's talent, as much as she struggles to hide it, is downright magical. Horses and other animals not only obey, but listen to her words. Daine, though, will have to learn to trust humans before she can come to terms with her powers, her past, and herself.
----------

      A Storm of Swords

By: 
George R.R. Martin

An alternate cover for this isbn can be found here.Here is the third volume in George R.R. Martin's magnificent cycle of novels that includes A Game of Thrones and A Clash of Kings. Together, this series comprises a genuine masterpiece of modern fantasy, destined to stand as one of the great achievements of imaginative fiction.Of the five contenders for power, one is dead, another in disfavor, and still the wars rage as alliances are made and broken. Joffrey sits on the Iron 


      Green Rider

By: 
Kristen Britain

On her long journey home from school after a fight which will surely lead to her expulsion, Karigan G'ladheon ponders her future as she trudges through the immense forest called Green Cloak. But her thoughts are interrupted by the clattering of hooves as a galloping horse bursts from the woods, the rider slumped over his mount's neck, impaled by two black-shafted arrows. As the young man lies dying on the road, he tells Karigan that he is a Green Rider, one of the legendary messengers of the king, and that he bears a "life and death" message for King Zachary. He begs Karigan to carry his message, warning her not to read it, and when she reluctantly agrees, he makes her swear on his sword to complete his mission "for love of country." As he bestows upon her the golden winged-horse brooch which is the symbol of his office, he whispers on his dying breath, "Beware the shadow man..." Karigan's promise changes her life forever. Pursued by unknown as


      The Magic of Recluce

By: 
L.E. Modesitt Jr. (Goodreads Author)

Young Lerris is dissatisfied with his life and trade, and yearns to find a place in the world better suited to his skills and temperament. But in Recluce a change in circumstances means taking one of two options: permanent exile from Recluce or the dangergeld, a complex, rule-laden wanderjahr in the lands beyond Recluce, with the aim of learning how the world works and what his place in it might be. Many do not survive. Lerris chooses dangergeld. When Lerris is sent into intensive training for his quest, it soon becomes clear that he has a natural talent for magic. And he will need magic in the lands beyond, where the power of the Chaos Wizards reigns unchecked. Though it goes against all of his instincts, Lerris must learn to use his powers in an orderly way before his wanderjahr, or fall prey to Chaos.
----------

      Dragon Prince

By: 
Melanie Rawn

Deadly dragons hold the secret to unimaginable wealth for mu

In [ ]:
loop_scrape('https://www.goodreads.com/list/show/11.Best_Crime_Mystery_Books')


      The Girl with the Dragon Tattoo

By: 
Stieg Larsson, 

Harriet Vanger, a scion of one of Sweden’s wealthiest families disappeared over forty years ago. All these years later, her aged uncle continues to seek the truth. He hires Mikael Blomkvist, a crusading journalist recently trapped by a libel conviction, to investigate. He is aided by the pierced and tattooed punk prodigy Lisbeth Salander. Together they tap into a vein of unfathomable iniquity and astonishing corruption.An international publishing sensation, Stieg Larsson’s The Girl with the Dragon Tattoo combines murder mystery, family saga, love story, and financial intrigue into one satisfyingly complex and entertainingly atmospheric novel.Alternate Cover Edition ISBN 0307269752 (ISBN13: 9780307269751)
----------

      And Then There Were None

By: 
Agatha Christie

First, there were ten—a curious assortment of strangers summoned as weekend guests to a private island off the coast of Devon. Their host, an eccentric millio


      Mystic River

By: 
Dennis Lehane (Goodreads Author)

When they were children, Sean Devine, Jimmy Marcus, and Dave Boyle were friends. But then a strange car pulled up to their street. One boy got into the car, two did not, and something terrible happened -- something that ended their friendship and changed all three boys forever. Twenty-five years later, Sean is a homicide detective. Jimmy is an ex-con who owns a corner store. And Dave is trying to hold his marriage together and keep his demons at bay -- demons that urge him to do terrible things. When Jimmy's daughter is found murdered, Sean is assigned to the case. His investigation brings him into conflict with Jimmy, who finds his old criminal impulses tempt him to solve the crime with brutal justice. And then there is Dave, who came home the night Jimmy's daughter died covered in someone else's blood. A tense and unnerving psychological thriller, Mystic River is also an epic novel of love and loyalty, faith and family, in w


      Presumed Innocent

By: 
Scott Turow (Goodreads Author)

Hailed as the most suspenseful and compelling novel in decades. Presumed Innocent brings to life our worst nightmare: that of an ordinary citizen facing conviction for the most terrible of all crimes. It's the stunning portrayal of one man's all-too-human, all-consuming fatal attraction for a passionate woman who is not his wife, and the story of how his obsession puts everything he loves and values on trial--including his own life. It's a book that lays bare a shocking world of betrayal and murder, as well as the hidden depths of the human heart. And it will hold you and haunt you...long after you have reached its shattering conclusion.--back cover
----------

      The Thirteenth Tale

By: 
Diane Setterfield (Goodreads Author)

All children mythologize their birth...So begins the prologue of reclusive author Vida Winter's collection of stories, which are as famous for the mystery of the missing thirteenth tale as they are


      1st to Die

By: 
James Patterson (Goodreads Author)

James Patterson, bestselling author of the Alex Cross novels Along Came a Spider, Kiss the Girls, and Pop Goes the Weasel, offers the first of a new series dubbed The Women's Murder Club, featuring a four-woman team that occasionally works outside the system. None of the gritty darkness or frenzied action is lost in 1st to Die, although the female protagonists offer an even deeper emotional context to this suspense thriller.Inspector Lindsay Boxer of the San Francisco Police Department suddenly finds herself in the middle of two horrifying situations: The first is that she's just learned she has an often-fatal blood disease. The second is a double homicide case she is now heading up that involves the murder of newlyweds on their wedding night. Burdened with Chris Raleigh, a new partner reassigned from the mayor's office, Lindsay finds that she has too much to deal with and turns to her best friend, Claire, the head ME on the c


      The Curious Incident of the Dog in the Night-Time

By: 
Mark Haddon

Christopher John Francis Boone knows all the countries of the world and their capitals and every prime number up to 7,057. He relates well to animals but has no understanding of human emotions. He cannot stand to be touched. And he detests the color yellow.Although gifted with a superbly logical brain, for fifteen-year-old Christopher everyday interactions and admonishments have little meaning. He lives on patterns, rules, and a diagram kept in his pocket. Then one day, a neighbor's dog, Wellington, is killed and his carefully constructive universe is threatened. Christopher sets out to solve the murder in the style of his favourite (logical) detective, Sherlock Holmes. What follows makes for a novel that is funny, poignant and fascinating in its portrayal of a person whose curse and blessing are a mind that perceives the world entirely literally.
----------

      The Historian

By: 
Elizabeth Kostova

Late on

...


      The Beach House

By: 
James Patterson (Goodreads Author), 

Jack Mullen is a driven student of the law. His brother Peter is a servant of the rich, parking the cars of the Hamptons' elite --- and perhaps satisfying their more intimate needs as well. Then Peter's body is found on the beach. Jack knows the drowning was no accident, but someone's unlimited power and money have bought the cops, the judges, the system. Now Jack is learning a lesson in justice he never got in law school ... and his astonishing plan to beat the billionaires will have you reeling --- and cheering --- to the very last page.
----------

      Factotum

By: 
Charles Bukowski

One of Charles Bukowski's best, this beer-soaked, deliciously degenerate novel follows the wanderings of aspiring writer Henry Chinaski across World War II-era America. Deferred from military service, Chinaski travels from city to city, moving listlessly from one odd job to another, always needing money but never badly enough to keep 


      Death on the Nile

By: 
Agatha Christie

The tranquillity of a cruise along the Nile is shattered by the discovery that Linnet Ridgeway has been shot through the head. She was young, stylish and beautiful, a girl who had everything - until she lost her life. Hercule Poirot recalls an earlier outburst by a fellow passenger: 'I'd like to put my dear little pistol against her head and just press the trigger.' Yet in this exotic setting, nothing is ever quite what it seems...
----------

      The Sculptress

By: 
Minette Walters

A WOMAN IMPRISONED FOR HIDEOUS SLAUGHTER. A WRITER TRAPPED IN A TRAGIC PAST. ONE HIDES THE TRUTH THE OTHER NEEDS.In prison, they call her the Sculptress for the strange figurines she carves - symbols of the day she hacked her mother and sister to pieces and reassembled them in a blood-drenched jigsaw. Sullen, menacing, grotesquely fat, Olive Martin is burned-out journalist Rosalind Leigh's only hope of getting a new book published.But as she interviews Oli


      Weep No More, My Lady

By: 
Mary Higgins Clark (Goodreads Author)

The New York Times calls Mary Higgins Clark's masterpiece "the ideal book for the beach bag, picnic hamper, or carry-on luggage." "Well done and very scary," says Cosmopolitan. Prepare yourself for the Queen of Suspense's "crackling tale of menace and love that holds your attention to the last page" (Andrew M. Greeley).Elizabeth Lange has arrived at Cypress Point Spa in Pebble Beach, California, weary of heart and soul. Still grieving for her beloved sister, a famous actress who plunged to her death from her Manhattan penthouse, Elizabeth is determined to unearth the truth about how Leila died. Dashing millionaire Ted Winters stands accused of her murder, but Elizabeth has doubts.Along the windswept cliffs of the Monterey coast, in luxurious bungalows, between gourmet meals and beachfront walks, uneasiness stalks Elizabeth while she begins opening doors to the past. As glimpses of the dark truth about Leila's lif


      The Man With a Load of Mischief

By: 
Martha Grimes (Goodreads Author)

At the Man with a Load of Mischief, they found the dead body stuck in a keg of beer. At the Jack and Hammer, another body was stuck out on the beam of the pub’s sign, replacing the mechanical man who kept the time. Two pubs. Two murders. One Scotland Yard inspector called in to help. Detective Chief Inspector Richard Jury arrives in Long Piddleton and finds everyone in the postcard village looking outside of town for the killer. Except for one Melrose Plant. A keen observer of human nature, he points Jury in the right direction: into the darkest parts of his neighbors’ hearts…
----------

      Digital Fortress

By: 
Dan Brown (Goodreads Author)

When the National Security Agency's invincible code-breaking machine encounters a mysterious code it cannot break, the agency calls its head cryptographer, Susan Fletcher, a brilliant and beautiful mathematician. What she uncovers sends shock waves through the corri


      Desecrating Solomon

By: 
Lucian Bane (Goodreads Author)

On the seventh day of the seventh year… she desecrates.Evil keeps a formidable schedule in Weston, West Virginia. Every seventh year, spirits from the town's abandoned insane asylum seek worthy souls to command. But this seventh year, a desecration unlike any before is scheduled to ravage the town. And Solomon Gorge is lured to the harrowing event by the screams of Silence.
----------

      The Neon Rain

By: 
James Lee Burke

Detective Dave Robicheaux has fought too many battles: in Vietnam, with killers and hustlers, with police brass, and with the bottle. Lost without his wife's love, Robicheaux's haunted soul mirrors the intensity and dusky mystery of New Orleans' French Quarter -- the place he calls home, and the place that nearly destroys him when he becomes involved in the case of a young prostitute whose body is found in a bayou. Thrust into the world of drug lords and arms smugglers, Robicheaux must face down a 

In [ ]:
loop_scrape('https://www.goodreads.com/list/show/135.Best_Horror_Novels')